# Submission Systems 
Submission system play an important role, if you want to develop your pygromos code. Many times, they are hidden in the Simulation_runner blocks. But maybe you want to develop something, where you need direct access on the submission system? 

This notebook will give you some examples, how you can use the submission systems.
Note that all submission systems are write in the same ways, such you can exchange them quickly.

In [1]:
from pygromos.simulations.hpc_queuing.submission_systems import local # this executes your code in your local session.
from pygromos.simulations.hpc_queuing.submission_systems import lsf # this module can be used to submit to the lsf-queue (e.g. on euler)
from pygromos.simulations.hpc_queuing.submission_systems import dummy # this is a dummy system, that only prints the commands
from pygromos.simulations.hpc_queuing.submission_systems.submission_job import Submission_job # this class stores all data for a single job

## Local Submission

This system executes the commands directly in your current session. This allows you to locally test or execute your code. Maybe if your process needs much more time, you want later to switch to a submission system for job-queueing.

In [2]:
sub_local = local.LOCAL()
sub_local.verbose = True

In [3]:
bash_command = "sleep 2; echo \"WUHA\"; sleep 2"
job = Submission_job(bash_command)

job_id = sub_local.submit_to_queue(job)
job_id

Submission Command: 	 . / j o b _ t e s t . s h


STDOUT: 
		b'WUHA\n'
END



0

In [4]:
#This is a dummy function, to not break the code!
sub_local.get_jobs_from_queue("FUN")

[]

## LSF Submission

The Lsf submission system allows to submit jobs to the IBM LSF-Queueing system.

**Careful! This part requires a running LSF-Queueing System on your System**

You can submit and kill jobs and arrays to the queue, as well as getting information from the queuing list.

In [5]:
#Construct system:
sub_lsf = lsf.LSF(nmpi=1, job_duration = "24:00", max_storage=100)
sub_lsf.verbose = True

sub_lsf._refresh_job_queue_list_all_s = 0 #you must wait at least 1s to update job_queue list

### Queue Checking:

In [6]:
sub_lsf.get_queued_jobs()
sub_lsf.job_queue_list

Exception: Could not get job_list!
err:
SubProcess Failed due to returncode: 127
 COMMAND: 
	bjobs -w
STDOUT:
	
STDERR:
	b'/bin/sh: 1: bjobs: not found\n'

### Submission:
here you can submit jobs to the queue as bash commands

In [7]:
bash_command = "sleep 5; echo \"WUHA\"; sleep 2"
job_name = "Test1"

job_id = sub_lsf.submit_to_queue(Submission_job(bash_command, job_name))

check queue


Exception: Could not get job_list!
err:
SubProcess Failed due to returncode: 127
 COMMAND: 
	bjobs -w
STDOUT:
	
STDERR:
	b'/bin/sh: 1: bjobs: not found\n'

In [8]:
#search for the just submitted job in the queue
sub_lsf.search_queue_for_jobid(job_id)

Exception: Could not get job_list!
err:
SubProcess Failed due to returncode: 127
 COMMAND: 
	bjobs -w
STDOUT:
	
STDERR:
	b'/bin/sh: 1: bjobs: not found\n'

In [9]:
sub_lsf.search_queue_for_jobname("Test1")

Exception: Could not get job_list!
err:
SubProcess Failed due to returncode: 127
 COMMAND: 
	bjobs -w
STDOUT:
	
STDERR:
	b'/bin/sh: 1: bjobs: not found\n'

### Submitting multiple jobs

In [10]:
bash_command = "sleep 2; echo \"WUHA\"; sleep 2"
job_ids = []
for test in range(3):
    job_name = "Test"+str(test)

    job_id = sub_lsf.submit_to_queue(Submission_job(bash_command, job_name))
    job_ids.append(job_id)

check queue


Exception: Could not get job_list!
err:
SubProcess Failed due to returncode: 127
 COMMAND: 
	bjobs -w
STDOUT:
	
STDERR:
	b'/bin/sh: 1: bjobs: not found\n'

In [11]:
sub_lsf.search_queue_for_jobname("Te", regex=True)

Exception: Could not get job_list!
err:
SubProcess Failed due to returncode: 127
 COMMAND: 
	bjobs -w
STDOUT:
	
STDERR:
	b'/bin/sh: 1: bjobs: not found\n'

### Killing a jobs

Remove a job the job queue

In [12]:
sub_lsf.kill_jobs(job_ids=[job_id])

Stopping: 0


ChildProcessError: could not execute this command: 
("SubProcess Failed due to returncode: 127\n COMMAND: \n\tbkill 0\nSTDOUT:\n\t\nSTDERR:\n\tb'/bin/sh: 1: bkill: not found\\n'",)

In [13]:
sub_lsf.search_queue_for_jobname("Te", regex=True)

Exception: Could not get job_list!
err:
SubProcess Failed due to returncode: 127
 COMMAND: 
	bjobs -w
STDOUT:
	
STDERR:
	b'/bin/sh: 1: bjobs: not found\n'